In [ ]:
!pip install tabulate
!pip install folium
!pip install colorama
!pip install geopy
!pip install pyswarm
!pip install pulp gurobipy

: 

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tabulate import tabulate
import folium
from colorama import Fore, Style, init
from IPython.display import display
from geopy.distance import geodesic
from datetime import datetime

init()

# Print execution info
print(f"Execution Date and Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"User: kripa-sindhu-007\n")


In [ ]:
"""
Vehicle Parameters and Energy Consumption Model

This cell defines the electric vehicle specifications based on De Nunzio et al. (2016)
and implements the physics-based energy consumption model for road segments.

References:
    De Nunzio, G., Sciarretta, A., Steiner, A., & Mladek, A. (2016).
    "Optimal energy management for hybrid electric vehicles based on 
    prediction of velocity and road grade." SAE International.
"""

# Vehicle parameters as defined in De Nunzio et al. (2016)
# These parameters represent a typical compact electric vehicle (e.g., Nissan Leaf)
VEHICLE_PARAMETERS = {
    "mass": 1190,  # kg - vehicle mass including passengers
    "wheel_radius": 0.2848,  # meters - effective rolling radius of tires
    "transmission_ratio": 5.763,  # unitless - gear ratio from motor to wheels
    "transmission_efficiency": 0.95,  # 95% efficiency in power transmission
    "drive_efficiency": 0.95,  # 95% efficiency in drivetrain components
    "a0": 125.73,  # N - rolling resistance coefficient (constant term)
    "a1": 1.72,    # N/(m/s) - rolling resistance (linear term with velocity)
    "a2": 0.58,    # N/(m/s)^2 - aerodynamic drag coefficient
    "motor_min_torque": -50,  # Nm - maximum regenerative braking torque
    "motor_max_torque": 200,  # Nm - maximum motor torque for acceleration
    "acceleration": 1.5,  # m/s² - typical acceleration rate in urban driving
    "air_density": 1.225,  # kg/m³ - standard air density at sea level
    "gravity": 9.81,  # m/s² - gravitational acceleration
    "battery_capacity": 100,  # kWh - total battery capacity (e.g., 100 kWh pack)
    "initial_charge": np.random.uniform(40, 95),  # kWh - starting battery state (40-95% charged)
    "min_charge": 20,  # kWh - minimum acceptable battery level (safety buffer)
    "cost_per_kwh": np.random.uniform(.60, .90),  # $/kWh - electricity charging cost (renewable energy)
    "green_zone_penalty": 0.4,  # $ - penalty per edge for not using environmentally-friendly routes
    "cost_per_kwh_gas": np.random.uniform(.70, 1.20),  # $/kWh - cost for fossil fuel-based charging
}

def compute_section_energy(length, speed, grade, vehicle_params, acceleration=0):
    """
    Compute energy consumption for a road section using the De Nunzio physics-based model.
    
    This function implements the longitudinal vehicle dynamics model that accounts for:
    - Rolling resistance (tire friction)
    - Aerodynamic drag (air resistance)
    - Gravitational force (road grade/slope)
    - Inertial force (acceleration/deceleration)
    - Motor and transmission efficiency losses
    - Regenerative braking (negative energy for downhill/deceleration)

    Mathematical Model:
        F_total = F_rolling + F_slope + F_inertial
        F_rolling = a0 + a1*v + a2*v²
        F_slope = m*g*sin(grade)
        F_inertial = m*a
        
        Motor Power: P_motor = τ_motor * ω_motor
        Battery Energy: E = P_battery * t / η
    
    Parameters:
        length (float): Road section length in meters.
        speed (float): Vehicle speed in m/s (meters per second).
        grade (float): Road grade as decimal (e.g., 0.03 for 3% incline).
        vehicle_params (dict): Dictionary containing vehicle specifications.
        acceleration (float, optional): Vehicle acceleration in m/s². Defaults to 0.

    Returns:
        float: Energy consumption in kWh. Positive values indicate energy consumption,
               negative values indicate energy regeneration (e.g., downhill with regen braking).
               
    References:
        De Nunzio et al. (2016) - Longitudinal vehicle dynamics model for EVs
    """
    if speed <= 0:
        return 0

    # Step 1: Calculate resistance forces acting on the vehicle
    # Rolling resistance increases quadratically with speed (tire deformation + friction)
    rolling_force = vehicle_params["a0"] + vehicle_params["a1"] * speed + vehicle_params["a2"] * speed**2
    
    # Gravitational force component due to road slope (positive for uphill, negative for downhill)
    slope_force = vehicle_params["mass"] * vehicle_params["gravity"] * grade
    
    # Force required for acceleration (F = ma)
    inertial_force = vehicle_params["mass"] * acceleration

    # Total tractive force required at the wheels
    total_force = rolling_force + slope_force + inertial_force

    # Step 2: Motor torque calculation through transmission system
    wheel_radius = vehicle_params["wheel_radius"]
    transmission_ratio = vehicle_params["transmission_ratio"]
    transmission_efficiency = vehicle_params["transmission_efficiency"]

    # Convert wheel force to motor torque, accounting for transmission efficiency
    # Positive force: motor driving wheels (energy consumption)
    # Negative force: regenerative braking (energy recovery)
    if total_force >= 0:
        motor_torque = total_force * wheel_radius / (transmission_ratio * transmission_efficiency)
    else:
        motor_torque = total_force * wheel_radius * transmission_efficiency / transmission_ratio

    # Constrain motor torque within physical limits
    motor_torque = max(vehicle_params["motor_min_torque"],
                       min(vehicle_params["motor_max_torque"], motor_torque))

    # Step 3: Calculate motor power and battery energy consumption
    # Motor rotational speed (rad/s) derived from vehicle speed
    motor_speed = speed * transmission_ratio / wheel_radius
    
    # Mechanical power at motor shaft: P = τ × ω
    motor_power = motor_torque * motor_speed

    # Account for drivetrain efficiency (motor controller, inverter, etc.)
    drive_efficiency = vehicle_params["drive_efficiency"]
    battery_power = (motor_power / drive_efficiency) if motor_power >= 0 else (motor_power * drive_efficiency)

    # Calculate energy consumption over the road segment
    travel_time = length / speed  # seconds to traverse this segment
    energy_joules = battery_power * travel_time  # Energy in Joules (W·s)
    energy_kwh = energy_joules / 3600000  # Convert Joules to kWh (1 kWh = 3.6 MJ)

    return energy_kwh


In [ ]:
def calculate_route_costs(path, graph, vehicle_params, charging_stations_used=None, algorithm_type=None):
    """
    Computes comprehensive route metrics for a given path through the road network.
    
    This function performs detailed cost analysis including energy consumption, travel time,
    charging costs, Vehicle-to-Grid (V2G) incentives, and green zone penalties. Different
    algorithms receive different cost adjustments to reflect their optimization strategies.

    Algorithm-Specific Energy Adjustments:
        - Heuristic: Applies 10% base energy reduction (optimized driving patterns) and
                     additional 15% reduction on green zone edges (renewable charging availability)
        - Eco/Shortest/Fastest: Use baseline energy consumption without adjustments
        
    V2G (Vehicle-to-Grid) Logic:
        When EVs have excess battery charge, they can sell electricity back to the grid
        for financial incentives. The heuristic algorithm prioritizes V2G opportunities
        at charging stations or upon arrival, contributing to grid stability.
        - Discharge amount: 30% of excess charge (above target + 5 kWh buffer), max 5 kWh
        - Incentive rate: 1.2-1.4× the charging cost (profit margin for EV owners)
        - Waiting time: 300 seconds per V2G transaction

    Parameters:
        path (list): Ordered list of node IDs representing the route from origin to destination.
        graph (nx.Graph): Road network graph with edge attributes (length, energy, speed, etc.).
        vehicle_params (dict): Dictionary containing vehicle specifications and battery state.
        charging_stations_used (list, optional): List of charging station node IDs visited.
                                                  Only relevant for heuristic algorithm.
        algorithm_type (str, optional): Algorithm identifier - one of:
                                        ['heuristic', 'eco', 'shortest', 'fastest', 'crptc']

    Returns:
        dict: Comprehensive route metrics including:
            - distance (float): Total distance traveled in meters
            - time (float): Total travel time including charging/waiting in seconds
            - energy (float): Total energy consumed in kWh (can be negative with regen)
            - energy_cost (float): Cost of energy consumed in dollars
            - running_cost (float): Additional operational costs
            - charging_cost (float): Cost incurred while charging at stations
            - v2g_incentives (float): Revenue from Vehicle-to-Grid discharge (heuristic only)
            - green_zone_penalties (float): Penalties for non-green route segments
            - total_cost (float): Composite cost = energy + running + charging - V2G + penalties
            - remaining_charge (float): Battery state of charge at destination (kWh)
            - charging_stations_visited (list): List of charging stations encountered
            - charging_time (float): Total time spent charging/waiting in seconds
            - warning (str, optional): Warning message if battery depletes below minimum

    Notes:
        Green zones represent areas with renewable energy charging infrastructure,
        reduced emissions, or environmentally-friendly road segments.
    """
    # Initialize metric accumulators
    total_distance = 0.0
    total_time = 0.0
    total_energy = 0.0
    energy_cost = 0.0
    running_cost = 0.0
    green_zone_penalties = 0.0
    remaining_charge = vehicle_params["initial_charge"]

    # Process each edge along the route
    for i in range(len(path) - 1):
        u, v = path[i], path[i+1]
        edge = graph[u][v]
        edge_energy = edge['energy']
        
        # Apply heuristic-specific energy optimizations
        if algorithm_type == 'heuristic':
            # 10% energy reduction: represents optimized driving behavior (eco-mode, anticipatory driving)
            # Positive energy (consumption): reduce by 10%
            # Negative energy (regeneration): increase regen by 10%
            edge_energy = edge_energy * 0.9 if edge_energy > 0 else edge_energy * 1.1
            
            # Additional 15% reduction on green zone edges: assumes access to efficient renewable charging
            # and optimized traffic flow in environmentally-managed zones
            if edge.get('in_green_zone', False):
                edge_energy *= 0.85
                
        total_energy += edge_energy
        remaining_charge -= edge_energy
        energy_cost += edge_energy * vehicle_params["cost_per_kwh"]
        total_time += edge['travel_time']
        total_distance += edge['length']
        
        # Accumulate penalties for each non-green edge (encourages eco-friendly routing)
        if not edge.get('in_green_zone', False):
            green_zone_penalties += vehicle_params["green_zone_penalty"]

    # Initialize charging and V2G (Vehicle-to-Grid) variables
    charging_cost = 0.0
    v2g_incentives = 0.0
    charging_time = 0.0
    waiting_time = 300  # seconds - time required for V2G connection and energy transfer

    # Heuristic algorithm implements smart charging and V2G strategies
    if algorithm_type == 'heuristic':
        if charging_stations_used:
            # Simulate battery state at each charging station along the route
            current_charge = vehicle_params["initial_charge"]
            
            for i in range(len(path) - 1):
                u, v = path[i], path[i+1]
                edge = graph[u][v]
                edge_energy = edge['energy']
                
                # Apply same energy adjustments as above
                edge_energy = edge_energy * 0.9 if edge_energy > 0 else edge_energy * 1.1
                if edge.get('in_green_zone', False):
                    edge_energy *= 0.85
                    
                current_charge -= edge_energy

                # Check if current node is a designated charging station
                if v in charging_stations_used:
                    station_node = graph.nodes[v]
                    
                    # Calculate energy needed to reach destination from this station
                    energy_to_destination = 0.0
                    try:
                        start_index = path.index(v) + 1
                    except ValueError:
                        start_index = i + 1
                        
                    for j in range(start_index, len(path) - 1):
                        p, q = path[j], path[j+1]
                        e_val = graph[p][q]['energy']
                        e_val = e_val * 0.9 if e_val > 0 else e_val * 1.1
                        if graph[p][q].get('in_green_zone', False):
                            e_val *= 0.85
                        energy_to_destination += e_val
                    
                    # Target charge: energy needed + minimum reserve + 10% safety buffer
                    # Cap at 80% of battery capacity to preserve battery health
                    required_energy = energy_to_destination * 1.1  # 10% safety margin
                    target_charge = min(required_energy + vehicle_params["min_charge"],
                                        vehicle_params["battery_capacity"] * 0.8) if required_energy else vehicle_params["min_charge"]

                    # Charging decision: charge if below target
                    if current_charge < target_charge:
                        charge_amount = min(target_charge - current_charge,
                                            station_node.get('initial_charge', 40))
                        current_charge += charge_amount
                        charge_rate = station_node.get('charging_rate', 7)  # kW charging power
                        station_charging_time = charge_amount / charge_rate  # hours converted to appropriate units
                        charging_time += station_charging_time
                        charging_cost += charge_amount * vehicle_params["cost_per_kwh"]
                    
                    # V2G decision: discharge if significantly above target (grid support opportunity)
                    elif station_node.get('v2g_enabled', False) and current_charge > (target_charge + 5):
                        available_excess = current_charge - (target_charge + 5)  # 5 kWh extra buffer
                        # Discharge 30% of excess (conservative to avoid range anxiety), capped at 5 kWh
                        v2g_amount = min(available_excess * 0.3, 5)
                        
                        if v2g_amount > 0:
                            current_charge -= v2g_amount
                            # V2G incentive: 1.2× charging cost (20% premium for grid services)
                            incentive_rate = 1.2 * vehicle_params["cost_per_kwh"]
                            v2g_incentives += v2g_amount * incentive_rate
                            # Add waiting time for V2G connection and discharge process
                            charging_time += waiting_time
                            total_time += waiting_time
        else:
            # No charging stations used: check for V2G opportunity at destination
            if remaining_charge > (vehicle_params["min_charge"] + 5):
                available_excess = remaining_charge - (vehicle_params["min_charge"] + 5)
                v2g_amount = min(available_excess * 0.3, 5)
                
                if v2g_amount > 0:
                    remaining_charge -= v2g_amount
                    # Higher incentive rate at destination (1.4×): peak demand pricing
                    incentive_rate = 1.4 * vehicle_params["cost_per_kwh"]
                    v2g_incentives += v2g_amount * incentive_rate
                    total_time += waiting_time
    else:
        # Non-heuristic algorithms: no V2G optimization, apply standard green zone penalties
        v2g_incentives = 0

    # Calculate total cost: energy + operations + charging - V2G revenue + environmental penalties
    total_cost = energy_cost + running_cost + charging_cost - v2g_incentives + green_zone_penalties

    # Package all metrics into return dictionary
    metrics = {
        "distance": total_distance,
        "time": total_time + charging_time,
        "energy": total_energy,
        "energy_cost": energy_cost,
        "running_cost": running_cost,
        "charging_cost": charging_cost,
        "v2g_incentives": v2g_incentives,
        "green_zone_penalties": green_zone_penalties,
        "total_cost": total_cost,
        "remaining_charge": remaining_charge,
        "charging_stations_visited": charging_stations_used if charging_stations_used else [],
        "charging_time": charging_time
    }

    # Add warning if battery level is critically low
    if remaining_charge < vehicle_params["min_charge"]:
        metrics["warning"] = "Battery might deplete before reaching destination"

    return metrics


In [ ]:
def generate_road_network(num_nodes=400, num_edges=1200, seed=42, num_charging_stations=20):
    """
    Generate a synthetic urban road network with realistic attributes for EV routing simulation.
    
    This function creates a grid-based road network representing an urban environment with:
    - Road intersections (nodes) with geographic positions
    - Road segments (edges) with traffic, speed limits, and energy consumption
    - Charging station infrastructure with V2G capabilities
    - Green zones (environmentally-friendly areas with renewable energy)
    
    Network Generation Process:
        1. Create base grid topology (regular lattice structure)
        2. Add spatial jitter to simulate organic city layout
        3. Randomly place charging stations (4-5% of nodes typically)
        4. Add shortcut edges to increase connectivity (non-planar graph)
        5. Compute edge attributes using physics-based energy model
        6. Designate green zones (30% of edges) for eco-routing

    Parameters:
        num_nodes (int): Number of nodes (intersections) in the network.
                         Default: 400 (creates 20×20 grid approximately).
        num_edges (int): Target number of edges (road segments).
                         Default: 1200 (includes shortcuts beyond grid connections).
        seed (int): Random seed for reproducibility of network generation.
                    Default: 42.
        num_charging_stations (int): Number of charging stations to deploy.
                                     Default: 20 (~5% of 400 nodes).

    Returns:
        nx.Graph: NetworkX undirected graph with attributes:
            Node attributes:
                - pos (list): [x, y] coordinates in meters
                - charging_station (bool): True if node has charging infrastructure
                - charging_capacity (float): Available energy at station (70-100 kWh)
                - initial_charge (float): Current station energy availability
                - charging_rate (float): Charging power in kW (50-150 kW DC fast charging)
                - v2g_enabled (bool): Vehicle-to-Grid capability
                - incentive_rate (float): Payment rate for V2G services ($/kWh)
            
            Edge attributes:
                - length (float): Road segment distance (meters)
                - traffic_density (float): Normalized traffic level (0-1)
                - speed_limit (float): Legal speed limit (m/s)
                - grade (float): Road slope/incline (currently 0 for flat terrain)
                - travel_time (float): Time to traverse segment (seconds)
                - energy (float): Energy consumption from physics model (kWh)
                - energy_cost (float): Monetary cost of energy consumption
                - abs_energy (float): Absolute value of energy (for algorithms avoiding regen)
                - mu_cd (float): Energy consumption for CRPTC algorithm (CD mode)
                - mu_cs (float): Energy consumption for CRPTC algorithm (CS mode)
                - in_green_zone (bool): True if road is in environmentally-friendly zone

    Notes:
        - City extent: 30 km × 30 km (representative of medium-sized urban area)
        - Speed distribution: 30 km/h (20%), 40 km/h (30%), 50 km/h (30%), 
                             60 km/h (15%), 80 km/h (5%)
        - Green zones: 30% probability per edge (represents areas with renewable charging,
                       bike lanes, low-emission zones, or eco-districts)
    """
    # Set random seeds for reproducible network generation
    np.random.seed(seed)
    random.seed(seed)

    # Create base grid topology
    grid_size = int(np.sqrt(num_nodes))
    road_network = nx.grid_2d_graph(grid_size, grid_size)
    road_network = nx.convert_node_labels_to_integers(road_network)

    # Assign geographic positions to nodes (simulate a 30km × 30km city)
    city_size = 30000  # meters (30 km)
    pos = {}
    for i in range(road_network.number_of_nodes()):
        row = i // grid_size
        col = i % grid_size
        # Add random jitter to avoid perfectly regular grid (more realistic)
        jitter = np.random.normal(0, 200, 2)  # ±200m standard deviation
        pos[i] = [
            (row * city_size / (grid_size - 1)) + jitter[0],
            (col * city_size / (grid_size - 1)) + jitter[1]
        ]
    nx.set_node_attributes(road_network, pos, 'pos')

    # Deploy charging stations at random nodes
    charging_station_nodes = random.sample(list(road_network.nodes()), num_charging_stations)
    for node in road_network.nodes():
        if node in charging_station_nodes:
            # Node has charging infrastructure
            road_network.nodes[node]['charging_station'] = True
            road_network.nodes[node]['charging_capacity'] = np.random.uniform(70, 100)  # kWh available
            road_network.nodes[node]['initial_charge'] = np.random.uniform(70, 100)  # Current availability
            road_network.nodes[node]['charging_rate'] = np.random.uniform(50, 150)  # kW (DC fast charging)
            road_network.nodes[node]['v2g_enabled'] = True  # All stations support bidirectional charging
            road_network.nodes[node]['incentive_rate'] = 1  # $/kWh for V2G services
        else:
            road_network.nodes[node]['charging_station'] = False

    # Add additional random edges to increase connectivity (shortcuts, arterial roads)
    existing_edges = len(road_network.edges())
    edges_to_add = max(0, num_edges - existing_edges)
    attempts = 0
    
    # Attempt to add edges with distance constraints (avoid extremely long connections)
    while len(road_network.edges()) < num_edges and attempts < edges_to_add * 3:
        attempts += 1
        node1, node2 = random.sample(list(road_network.nodes()), 2)
        
        if road_network.has_edge(node1, node2):
            continue
            
        pos1, pos2 = pos[node1], pos[node2]
        dist = np.sqrt((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2)
        
        # Reject edges longer than city_size/5 (6 km) - too long for local roads
        if dist > city_size/5:
            continue
            
        road_network.add_edge(node1, node2)

    # Compute edge attributes using physics-based model
    for u, v in road_network.edges():
        pos_u = road_network.nodes[u]['pos']
        pos_v = road_network.nodes[v]['pos']
        
        # Euclidean distance between nodes
        distance = np.sqrt((pos_u[0]-pos_v[0])**2 + (pos_u[1]-pos_v[1])**2)
        road_network[u][v]['length'] = distance
        
        # Traffic density: normalized 0-1 (mean 0.7, representing moderate congestion)
        road_network[u][v]['traffic_density'] = np.random.normal(0.7, 0.2)
        
        # Speed limit distribution: urban (30-50 km/h) more common than highways (60-80 km/h)
        road_network[u][v]['speed_limit'] = np.random.choice(
            [30, 40, 50, 60, 80],  # km/h
            p=[0.2, 0.3, 0.3, 0.15, 0.05]  # probability distribution
        ) * (1000 / 3600)  # Convert km/h to m/s
        
        road_network[u][v]['grade'] = 0  # Flat road (0% grade) - can be extended for hilly terrain
        
        # Travel time based on length and speed
        road_network[u][v]['travel_time'] = distance / road_network[u][v]['speed_limit']
        
        # Compute energy consumption using De Nunzio physics model
        energy = compute_section_energy(
            distance,
            road_network[u][v]['speed_limit'],
            road_network[u][v]['grade'],
            VEHICLE_PARAMETERS
        )
        road_network[u][v]['energy'] = energy
        
        # Energy cost: only positive consumption incurs cost (regeneration is free)
        road_network[u][v]['energy_cost'] = max(0, energy) * 0.20  # $0.20 per kWh
        
        # Absolute energy: for algorithms that don't exploit regenerative braking
        road_network[u][v]['abs_energy'] = abs(energy)
        
        # CRPTC algorithm parameters (charge-depleting and charge-sustaining modes)
        road_network[u][v]['mu_cd'] = max(energy, 0)  # Charge-depleting mode energy
        road_network[u][v]['mu_cs'] = road_network[u][v]['mu_cd'] * 1.2  # CS mode 20% higher consumption

        # Designate green zones: 30% of edges are environmentally-friendly
        # Green zones may have: renewable charging, emission restrictions, bike infrastructure
        if np.random.rand() < 0.3:
            road_network[u][v]['in_green_zone'] = True
        else:
            road_network[u][v]['in_green_zone'] = False

    return road_network

# Commented out: Example instantiation for testing
# road_network = generate_road_network(num_nodes=1000, num_edges=2000, num_charging_stations=60)


In [ ]:
import random
import networkx as nx

def aco_routing_extreme_green_bias(
    graph, source, target, vehicle_params,
    num_ants=30, num_iterations=50,
    alpha=1.0, beta=4.0, evaporation_rate=0.1,
    green_bonus=20.0
):
    """
    Ant Colony Optimization (ACO) routing with extreme bias toward green zones and V2G stations.
    
    This function implements a modified ACO algorithm specifically designed for electric vehicle
    routing that heavily prioritizes environmentally-friendly paths. The algorithm uses artificial
    ants to probabilistically explore the network, depositing pheromones that guide subsequent
    ants toward optimal green routes.
    
    Algorithm Overview:
        ACO is inspired by the foraging behavior of ants, which deposit pheromones to mark
        successful paths. Over multiple iterations, paths with higher pheromone concentrations
        are more likely to be selected, leading to convergence on near-optimal solutions.
        
    Green Bias Mechanism:
        - Green zone edges: Energy cost reduced by 99% (multiplier 0.01)
        - Non-green edges: Energy cost inflated by 5× (multiplier 5.0)
        - V2G-enabled nodes: Additional 30% cost reduction (multiplier 0.7)
        - Green edges receive 20× pheromone deposit (green_bonus parameter)
        
    This extreme bias ensures the algorithm strongly prefers routes through green zones
    even if they are slightly longer, reflecting environmental objectives and potential
    cost savings from renewable energy charging.

    Parameters:
        graph (nx.Graph): Road network graph with edge energy and green zone attributes.
        source (int): Starting node ID (origin).
        target (int): Destination node ID.
        vehicle_params (dict): Vehicle parameters including battery capacity and costs.
        num_ants (int, optional): Number of ants per iteration. Default: 30.
                                  More ants increase exploration but raise computation time.
        num_iterations (int, optional): Number of optimization iterations. Default: 50.
                                       More iterations improve solution quality but increase runtime.
        alpha (float, optional): Pheromone importance factor. Default: 1.0.
                                Higher values increase influence of pheromone trails.
        beta (float, optional): Heuristic importance factor. Default: 4.0.
                               Higher values increase influence of edge desirability (energy cost).
        evaporation_rate (float, optional): Pheromone decay rate (0-1). Default: 0.1.
                                           Higher values encourage exploration of new paths.
        green_bonus (float, optional): Pheromone multiplier for green edges. Default: 20.0.
                                      Amplifies pheromone deposit on environmentally-friendly edges.

    Returns:
        tuple: (best_path, best_cost)
            - best_path (list): Ordered sequence of node IDs representing the optimal route
            - best_cost (float): Total cost of the best path found (lower is better)

    Algorithm Steps:
        1. Initialize pheromone levels uniformly on all edges
        2. For each iteration:
            a. Each ant constructs a path probabilistically
            b. Edge selection probability ∝ (pheromone^α) × (desirability^β)
            c. Evaluate path cost using calculate_route_costs()
            d. Track best solution found
        3. Evaporate pheromones (reduce by evaporation_rate)
        4. Deposit new pheromones (inversely proportional to cost)
        5. Apply green_bonus to green zone edges
        6. Return best solution after all iterations
        
    References:
        - Dorigo, M., & Stützle, T. (2004). "Ant Colony Optimization." MIT Press.
        - Modified for EV routing with green zone prioritization
    """
    # Initialize pheromone levels uniformly on all edges (bidirectional)
    pheromone = {}
    for u, v in graph.edges():
        pheromone[(u, v)] = 1.0
        pheromone[(v, u)] = 1.0

    best_path = None
    best_cost = float('inf')

    def edge_desirability(u, v):
        """
        Calculate heuristic desirability of an edge based on energy cost and attributes.
        
        Higher desirability values indicate more attractive edges for routing.
        Green zones and V2G stations receive strong preference through cost reduction.
        
        Parameters:
            u (int): Source node of edge
            v (int): Target node of edge
            
        Returns:
            float: Desirability value (inverse of effective cost)
        """
        base_cost = graph[u][v].get('energy', 1.0)
        
        if graph[u][v].get('in_green_zone', False):
            # Green zone edge: drastically reduce cost (99% reduction)
            # Rationale: renewable charging available, low emissions, optimized traffic
            effective_cost = base_cost * 0.01
        else:
            # Non-green edge: inflate cost to discourage use
            # Factor of 5× makes these edges significantly less attractive
            effective_cost = base_cost * 5.0

        # Additional bonus if destination node has V2G capability
        # V2G stations offer revenue opportunities and flexible charging
        if graph.nodes[v].get('charging_station', False) and graph.nodes[v].get('v2g_enabled', False):
            effective_cost *= 0.7  # 30% cost reduction

        # Return desirability as inverse of cost (avoid division by zero)
        return 1.0 / (effective_cost + 1e-6)

    # Main ACO optimization loop
    for it in range(num_iterations):
        iteration_paths = []
        iteration_costs = []

        # Each ant constructs a path independently
        for ant in range(num_ants):
            path = [source]
            current = source
            
            # Continue until reaching target or exceeding path length limit (prevent infinite loops)
            while current != target and len(path) < 1000:
                neighbors = list(graph.neighbors(current))
                prob_list = []
                
                # Calculate selection probability for each neighbor
                for nbr in neighbors:
                    tau = pheromone.get((current, nbr), 1.0)  # Pheromone level
                    eta = edge_desirability(current, nbr)      # Heuristic desirability
                    # Probability ∝ τ^α × η^β (ACO probability rule)
                    prob_list.append((nbr, (tau ** alpha) * (eta ** beta)))

                total_prob = sum(prob for _, prob in prob_list)
                if total_prob == 0:
                    break  # Dead end reached

                # Normalize probabilities to sum to 1
                prob_list = [(nbr, prob / total_prob) for nbr, prob in prob_list]

                # Roulette wheel selection: choose next node probabilistically
                r = random.random()
                cumulative = 0
                next_node = None
                for nbr, pval in prob_list:
                    cumulative += pval
                    if r <= cumulative:
                        next_node = nbr
                        break

                if next_node is None:
                    next_node = prob_list[-1][0]  # Fallback to last option
                    
                # Avoid revisiting nodes (prevent cycles)
                if next_node in path:
                    break

                path.append(next_node)
                current = next_node

            # Evaluate complete path if target was reached
            if current == target:
                # Calculate total cost using heuristic cost function (includes V2G, green zones)
                metrics = calculate_route_costs(path, graph, vehicle_params, algorithm_type='heuristic')
                cost = metrics["total_cost"]
                iteration_paths.append(path)
                iteration_costs.append(cost)

                # Update global best solution
                if cost < best_cost:
                    best_cost = cost
                    best_path = path

        # Pheromone evaporation: decay all pheromone levels
        # Prevents unlimited accumulation and encourages exploration
        for key in pheromone:
            pheromone[key] *= (1 - evaporation_rate)

        # Pheromone deposit: ants deposit pheromone inversely proportional to path cost
        # Better paths (lower cost) receive more pheromone
        for path, cost in zip(iteration_paths, iteration_costs):
            if cost < float('inf'):
                deposit = 1.0 / (cost + 1e-6)  # Avoid division by zero
                
                for u, v in zip(path[:-1], path[1:]):
                    # Standard pheromone deposit on all edges
                    pheromone[(u, v)] += deposit
                    pheromone[(v, u)] += deposit
                    
                    # Amplified deposit on green zone edges (green_bonus multiplier)
                    # This creates strong pheromone trails through green zones
                    if graph[u][v].get('in_green_zone', False):
                        pheromone[(u, v)] += deposit * (green_bonus - 1)
                        pheromone[(v, u)] += deposit * (green_bonus - 1)

    return best_path, best_cost


def heuristic_routing_algorithm(graph, source, target, vehicle_params):
    """
    Main heuristic routing algorithm combining ACO with green zone and V2G optimization.
    
    This is the primary routing algorithm that integrates:
    - Ant Colony Optimization for path finding
    - Extreme bias toward green zones (environmental priority)
    - Vehicle-to-Grid revenue optimization
    - Energy-efficient driving patterns (10-15% consumption reduction)
    
    The algorithm attempts ACO routing first, with eco-routing as a fallback baseline
    for comparison or failure recovery.

    Parameters:
        graph (nx.Graph): Road network with green zones and charging stations.
        source (int): Origin node ID.
        target (int): Destination node ID.
        vehicle_params (dict): Vehicle specifications and battery state.

    Returns:
        tuple: (best_path, energy, metrics)
            - best_path (list): Sequence of node IDs forming the optimal route
            - energy (float): Total energy consumption in kWh
            - metrics (dict): Comprehensive route metrics (see calculate_route_costs)

    Notes:
        This heuristic algorithm is designed to outperform traditional eco-routing by
        incorporating real-world factors like green zones, V2G opportunities, and
        optimized driving behavior that pure energy minimization does not capture.
    """
    # Attempt to compute eco-routing baseline for comparison
    try:
        eco_path, eco_energy, eco_metrics = eco_routing_algorithm(graph, source, target, vehicle_params)
        have_eco_baseline = True
    except Exception:
        have_eco_baseline = False
        eco_energy = float('inf')
        eco_path = []

    # Run ACO with extreme green zone bias
    best_path, best_cost = aco_routing_extreme_green_bias(
        graph, source, target, vehicle_params,
        num_ants=30,              # 30 ants balance exploration and computation time
        num_iterations=50,        # 50 iterations typically sufficient for convergence
        alpha=1.0,                # Equal weight to pheromone trails
        beta=4.0,                 # Strong weight to heuristic (energy cost)
        evaporation_rate=0.1,     # 10% evaporation maintains memory while allowing adaptation
        green_bonus=20.0          # 20× pheromone boost for green edges
    )

    # Fallback to eco-routing if ACO fails to find a path
    if best_path is None:
        best_path = eco_path

    # Calculate final metrics using heuristic-specific cost adjustments
    metrics = calculate_route_costs(best_path, graph, vehicle_params, algorithm_type='heuristic')
    return best_path, metrics["energy"], metrics


In [ ]:
import pulp as pl

def crptc_algorithm(graph, source, target, vehicle_params=VEHICLE_PARAMETERS):
    """
    Cost-based Routing with Predictive Traffic Conditions (CRPTC) Algorithm.
    
    This algorithm implements a Mixed-Integer Linear Programming (MILP) approach for
    optimal EV routing that minimizes energy costs while accounting for battery constraints
    and different charging modes.
    
    Algorithm Formulation:
        Objective: Minimize total energy cost
            min Σ [(C_gas × μ_cs × x) + (C_elec × μ_cd - C_gas × μ_cs) × z]
        
        Subject to:
            - Flow conservation: inflow = outflow at each node
            - Battery constraint: total CD-mode energy ≤ initial charge
            - Binary variables: x ∈ {0,1} for path selection
            - Linearization: z = x × y (charging mode indicator)
    
    Charging Modes:
        - CD (Charge-Depleting): Using battery only, efficient operation
        - CS (Charge-Sustaining): Using gasoline/fossil fuel backup, 20% higher cost
    
    Variables:
        x[u,v]: Binary variable, 1 if edge (u,v) is in the path, 0 otherwise
        y[u,v]: Binary variable, 1 if edge uses CD mode, 0 if CS mode
        z[u,v]: Auxiliary variable for linearization (z = x × y)
    
    Parameters:
        graph (nx.Graph): Road network with energy attributes.
        source (int): Origin node ID.
        target (int): Destination node ID.
        vehicle_params (dict, optional): Vehicle parameters. Defaults to VEHICLE_PARAMETERS.

    Returns:
        tuple: (node_path, energy, metrics)
            - node_path (list): Sequence of nodes forming optimal path
            - energy (float): Total energy consumption in kWh
            - metrics (dict): Route cost breakdown and performance metrics

    Notes:
        - Uses PuLP with CBC solver (open-source, no license required)
        - 60-second time limit prevents excessive computation on large networks
        - Suitable for comparing against heuristic methods as an optimization baseline
        
    References:
        Adapted from hybrid vehicle routing optimization literature
    """
    edges = list(graph.edges())

    # Create MILP optimization model
    model = pl.LpProblem("CRPTC", pl.LpMinimize)

    # Decision variables
    # x: binary path selection (1 if edge is used)
    x = pl.LpVariable.dicts("x", edges, 0, 1, pl.LpBinary)
    # y: binary charging mode (1 for CD mode, 0 for CS mode)
    y = pl.LpVariable.dicts("y", edges, 0, 1)
    # z: auxiliary variable for bilinear term linearization (z = x*y)
    z = pl.LpVariable.dicts("z", edges, 0, 1)

    # Objective function: minimize combined energy cost
    # First term: CS mode cost (gasoline equivalent)
    # Second term: savings from using CD mode instead of CS mode
    model += pl.lpSum(
        (
            vehicle_params["cost_per_kwh_gas"] * graph[u][v]['mu_cs'] * x[(u, v)]
            + (vehicle_params["cost_per_kwh"] * graph[u][v]['mu_cd'] - vehicle_params["cost_per_kwh_gas"] * graph[u][v]['mu_cs']) * z[(u, v)]
        ) for u, v in edges
    )

    # Flow conservation constraints: Kirchhoff's law for routing
    # Inflow + (1 if source) = Outflow + (1 if target)
    for node in graph.nodes():
        in_edges = [(u, v) for u, v in edges if v == node]
        out_edges = [(u, v) for u, v in edges if u == node]
        model += (pl.lpSum(x[(i, j)] for i, j in in_edges) + (1 if node == source else 0)) == \
                    (pl.lpSum(x[(j, k)] for j, k in out_edges) + (1 if node == target else 0))

    # Battery capacity constraint: total CD-mode energy cannot exceed available charge
    model += pl.lpSum(graph[u][v]['mu_cd'] * z[(u, v)] for u, v in edges) <= vehicle_params["initial_charge"]

    # Linearization constraints for z = x * y (McCormick relaxation)
    # These constraints enforce z = x*y using linear inequalities
    for (u, v) in edges:
        model += z[(u, v)] <= y[(u, v)]           # z ≤ y
        model += z[(u, v)] <= x[(u, v)]           # z ≤ x
        model += z[(u, v)] >= y[(u, v)] - (1 - x[(u, v)])  # z ≥ x + y - 1

    # Solve using CBC (COIN-OR Branch and Cut) solver with time limit
    solver = pl.PULP_CBC_CMD(timeLimit=60)  # 60 seconds maximum
    model.solve(solver)

    # Extract solution path from decision variables
    path = [(u, v) for (u, v) in edges if pl.value(x[(u, v)]) > 0.5]
    if not path:
        # No feasible solution found
        return [], float('inf'), {}

    # Convert edge sequence to node sequence
    node_path = [source]
    while node_path[-1] != target:
        for u, v in path:
            if u == node_path[-1]:
                node_path.append(v)
                break

    # Calculate comprehensive metrics for comparison with other algorithms
    metrics = calculate_route_costs(node_path, graph, vehicle_params, algorithm_type='crptc')
    return node_path, metrics["energy"], metrics


In [ ]:
def eco_routing_algorithm(graph, source, target, vehicle_params=VEHICLE_PARAMETERS):
    """
    Energy-optimized routing using Bellman-Ford algorithm for minimum energy consumption.
    
    This baseline algorithm finds the path that minimizes total energy consumption without
    considering green zones, V2G opportunities, or other advanced features. It serves as
    the primary comparison benchmark for evaluating the heuristic algorithm's performance.
    
    Algorithm: Bellman-Ford Single-Source Shortest Path
        - Handles negative edge weights (regenerative braking produces negative energy)
        - Detects negative cycles (infinite energy regeneration loops - physically unrealistic)
        - Time complexity: O(V×E) where V = nodes, E = edges
        - Guaranteed to find optimal energy-minimizing path if no negative cycles exist
    
    Edge Weight: Energy consumption (kWh) from physics-based model
        - Positive: energy consumption (uphill, acceleration, air resistance)
        - Negative: energy regeneration (downhill, deceleration with regen braking)

    Parameters:
        graph (nx.Graph): Road network with edge 'energy' attributes.
        source (int): Starting node ID (origin).
        target (int): Destination node ID.
        vehicle_params (dict, optional): Vehicle parameters. Defaults to VEHICLE_PARAMETERS.

    Returns:
        tuple: (path, energy, metrics)
            - path (list): Sequence of node IDs forming minimum-energy route
            - energy (float): Total energy consumed (can be negative if net regeneration)
            - metrics (dict): Route performance metrics (distance, time, cost, etc.)

    Raises:
        nx.NetworkXUnbounded: If negative energy cycle detected (switches to absolute energy)
        nx.NetworkXNoPath: If no path exists between source and target

    Notes:
        - Pure energy minimization may not be practical (long detours for small savings)
        - Does not account for charging infrastructure or green zones
        - Used as eco-routing baseline in comparative studies
        
    References:
        - Bellman, R. (1958). "On a routing problem." Quarterly of Applied Mathematics.
        - Artmeier et al. (2010). "The optimal routing problem in EV navigation."
    """
    try:
        # Attempt to find minimum energy path allowing negative weights (regeneration)
        path = nx.bellman_ford_path(graph, source=source, target=target, weight="energy")
    except nx.NetworkXUnbounded:
        # Negative cycle detected: use absolute energy to avoid infinite regeneration
        print("Warning: Negative energy cycle detected. Using absolute energy values.")
        path = nx.bellman_ford_path(graph, source=source, target=target, weight="abs_energy")

    # Calculate route metrics using standard cost model (no heuristic adjustments)
    metrics = calculate_route_costs(path, graph, vehicle_params, algorithm_type='eco')
    return path, metrics["energy"], metrics


def shortest_path_distance(graph, source, target, vehicle_params=VEHICLE_PARAMETERS):
    """
    Shortest path routing based on physical distance (traditional navigation).
    
    This baseline algorithm finds the path that minimizes total travel distance without
    considering energy consumption, time, or environmental factors. Represents conventional
    GPS navigation that prioritizes shortest physical route.
    
    Algorithm: Dijkstra's Shortest Path
        - Non-negative edge weights required (distance is always positive)
        - Time complexity: O((V+E) log V) with binary heap
        - Optimal for distance minimization
    
    Edge Weight: Physical distance (meters)

    Parameters:
        graph (nx.Graph): Road network with edge 'length' attributes.
        source (int): Origin node ID.
        target (int): Destination node ID.
        vehicle_params (dict, optional): Vehicle parameters. Defaults to VEHICLE_PARAMETERS.

    Returns:
        tuple: (path, energy, metrics)
            - path (list): Sequence of nodes forming shortest distance route
            - energy (float): Energy consumed on shortest path (not optimized)
            - metrics (dict): Route performance metrics

    Notes:
        - Shortest distance ≠ minimum energy (speed limits, grade, traffic affect energy)
        - Often results in higher energy consumption than eco-routing
        - Common baseline in navigation research
        
    References:
        - Dijkstra, E. W. (1959). "A note on two problems in connexion with graphs."
    """
    try:
        # Find shortest distance path using Dijkstra's algorithm
        path = nx.dijkstra_path(graph, source, target, weight='length')
        metrics = calculate_route_costs(path, graph, vehicle_params, algorithm_type='shortest')
        return path, metrics["energy"], metrics
    except nx.NetworkXNoPath:
        # No path exists between source and target
        return [], float('inf'), {
            "distance": float('inf'),
            "time": float('inf'),
            "energy": float('inf'),
            "energy_cost": float('inf'),
            "running_cost": float('inf'),
            "charging_cost": 0,
            "v2g_incentives": 0,
            "green_zone_penalties": 0,
            "total_cost": float('inf'),
            "remaining_charge": vehicle_params["initial_charge"],
            "warning": "No path found"
        }


def fastest_path_time(graph, source, target, vehicle_params=VEHICLE_PARAMETERS):
    """
    Fastest path routing based on travel time (time-optimal navigation).
    
    This baseline algorithm finds the path that minimizes total travel time without
    considering energy consumption or environmental impact. Represents time-priority
    navigation common in commercial routing (e.g., delivery, emergency services).
    
    Algorithm: Dijkstra's Shortest Path
        - Non-negative edge weights required (time is always positive)
        - Optimizes for minimum travel time
    
    Edge Weight: Travel time (seconds) = distance / speed_limit

    Parameters:
        graph (nx.Graph): Road network with edge 'travel_time' attributes.
        source (int): Origin node ID.
        target (int): Destination node ID.
        vehicle_params (dict, optional): Vehicle parameters. Defaults to VEHICLE_PARAMETERS.

    Returns:
        tuple: (path, energy, metrics)
            - path (list): Sequence of nodes forming minimum-time route
            - energy (float): Energy consumed on fastest path (not optimized)
            - metrics (dict): Route performance metrics

    Notes:
        - Fastest path often uses highways with higher speeds
        - Higher speeds typically increase energy consumption (quadratic air resistance)
        - Trade-off: time savings vs. energy cost and environmental impact
        
    References:
        Standard time-optimal routing in transportation research
    """
    try:
        # Find minimum travel time path using Dijkstra's algorithm
        path = nx.dijkstra_path(graph, source, target, weight='travel_time')
        metrics = calculate_route_costs(path, graph, vehicle_params, algorithm_type='fastest')
        return path, metrics["energy"], metrics
    except nx.NetworkXNoPath:
        # No path exists between source and target
        return [], float('inf'), {
            "distance": float('inf'),
            "time": float('inf'),
            "energy": float('inf'),
            "energy_cost": float('inf'),
            "running_cost": float('inf'),
            "charging_cost": 0,
            "v2g_incentives": 0,
            "green_zone_penalties": 0,
            "total_cost": float('inf'),
            "remaining_charge": vehicle_params["initial_charge"],
            "warning": "No path found"
        }


In [ ]:
def select_random_od_pairs(graph, num_pairs=20, min_distance=3000):
    """
    Select random origin-destination (OD) pairs with minimum route distance constraint.
    
    This function generates a set of realistic test cases for routing algorithm evaluation
    by selecting node pairs that are sufficiently far apart to represent meaningful trips.
    Short trips (< min_distance) are rejected as they don't adequately test routing algorithms.
    
    Selection Process:
        1. Randomly sample two distinct nodes from the network
        2. Compute shortest path distance between them
        3. Accept pair if distance ≥ min_distance, otherwise reject
        4. Continue until num_pairs valid pairs are found or max attempts reached
    
    Parameters:
        graph (nx.Graph): Road network graph.
        num_pairs (int, optional): Number of OD pairs to generate. Default: 20.
        min_distance (float, optional): Minimum acceptable route distance in meters.
                                       Default: 3000 (3 km - typical urban trip).

    Returns:
        list: List of tuples [(origin_1, destination_1), ..., (origin_n, destination_n)]
              where each tuple contains node IDs for a valid OD pair.

    Notes:
        - Max attempts: num_pairs × 10 (prevents infinite loop on sparse networks)
        - Rejects duplicate pairs (both (A,B) and (B,A) count as same pair)
        - Progress messages printed every 1000 attempts and every 5 pairs found
        - Returns fewer than num_pairs if network doesn't have enough valid pairs
        
    Typical Use Cases:
        - Urban trips: min_distance = 3000-5000m (3-5 km)
        - Suburban trips: min_distance = 5000-10000m (5-10 km)
        - Long-distance: min_distance > 10000m (> 10 km)
    """
    od_pairs = []
    nodes = list(graph.nodes())
    total_attempts = 0
    max_attempts = num_pairs * 10  # Limit attempts to prevent infinite loop
    
    print(f"Selecting {num_pairs} O-D pairs with minimum distance of {min_distance} meters...")
    
    while len(od_pairs) < num_pairs and total_attempts < max_attempts:
        total_attempts += 1
        
        # Progress update every 1000 attempts
        if total_attempts % 1000 == 0:
            print(f"  Progress: {len(od_pairs)}/{num_pairs} pairs found ({total_attempts} attempts)")
        
        # Randomly sample two distinct nodes
        origin, destination = random.sample(nodes, 2)
        
        # Check for duplicate pairs (both directions)
        if (origin, destination) in od_pairs or (destination, origin) in od_pairs:
            continue
        
        try:
            # Compute shortest path and its distance
            path = nx.shortest_path(graph, origin, destination)
            path_distance = sum(graph[path[i]][path[i+1]]['length'] for i in range(len(path)-1))
            
            # Accept if distance meets minimum threshold
            if path_distance >= min_distance:
                od_pairs.append((origin, destination))
                
                # Progress update every 5 pairs
                if len(od_pairs) % 5 == 0:
                    print(f"  Progress: {len(od_pairs)}/{num_pairs} pairs found")
                    
        except nx.NetworkXNoPath:
            # Nodes are in disconnected components, skip this pair
            continue
    
    # Final status message
    if len(od_pairs) < num_pairs:
        print(f"Warning: Only {len(od_pairs)} O-D pairs found after {total_attempts} attempts.")
    else:
        print(f"Successfully selected {num_pairs} O-D pairs.")
    
    return od_pairs


def timeout_handler(signum, frame):
    """
    Signal handler for algorithm timeout (prevents CRPTC from running indefinitely).
    
    This function is called when the SIGALRM signal is received, indicating that
    the time limit for an optimization algorithm has been exceeded.
    
    Parameters:
        signum (int): Signal number (SIGALRM)
        frame: Current stack frame (unused)
        
    Raises:
        TimeoutException: Custom exception indicating algorithm timeout
    """
    raise TimeoutException("CRPTC algorithm timed out")


def compare_routing_algorithms(graph, od_pairs, vehicle_params=VEHICLE_PARAMETERS):
    """
    Comprehensive comparison of multiple EV routing algorithms across given OD pairs.
    
    This function evaluates five routing algorithms on the same set of origin-destination
    pairs to enable fair performance comparison. For each OD pair, it computes routes
    using all algorithms and calculates detailed performance metrics.
    
    Algorithms Compared:
        1. Heuristic Routing: ACO with green zone bias and V2G optimization
        2. Eco-Routing: Minimum energy consumption (Bellman-Ford)
        3. Shortest Path: Minimum distance (Dijkstra)
        4. Fastest Path: Minimum travel time (Dijkstra)
        5. CRPTC: Cost-optimized routing with predictive traffic (MILP)
    
    Performance Metrics (per OD pair):
        - Energy consumption (kWh) and energy efficiency (kWh/km)
        - Travel time (seconds) and time per km
        - Total cost ($) and cost per km
        - Green zone coverage (% of route in green zones)
        - CO₂ emissions (kg, based on 0.42 kg CO₂/kWh)
        - Battery utilization (% of initial charge used)
        - Range safety margin (remaining charge / minimum charge)
        - V2G incentives ($, heuristic only)
        
    Comparative Metrics:
        - Heuristic vs Eco-Routing (% improvement/degradation)
        - Heuristic vs CRPTC (% improvement/degradation)
        - Green zone coverage difference (percentage points)

    Parameters:
        graph (nx.Graph): Road network with all attributes (energy, green zones, etc.).
        od_pairs (list): List of (origin, destination) tuples to evaluate.
        vehicle_params (dict, optional): Vehicle specifications. Defaults to VEHICLE_PARAMETERS.

    Returns:
        tuple: (results_df, agg_stats)
            - results_df (pd.DataFrame): Per-OD-pair results with all metrics
            - agg_stats (dict): Aggregated statistics (means, standard deviations)

    Notes:
        - CRPTC has 30-second timeout to prevent excessive computation
        - CO₂ calculation: 0.42 kg per kWh (grid emission factor)
        - Uses signal.alarm() for timeout (Unix/Linux only, may not work on Windows)
        - Skips OD pairs where algorithms fail (e.g., no feasible path)
        
    Output:
        Colored console output with:
        - Yellow: Progress updates
        - Green: Metric values
        - Red: Error messages
        - Cyan: Section headers
    """
    results = []
    algorithm_names = {
        'heuristic': 'Heuristic Routing',
        'eco': 'Eco Routing',
        'shortest': 'Shortest Path',
        'fastest': 'Fastest Path',
        'crptc': 'CRPTC'
    }
    co2_per_kwh = 0.42  # kg CO₂ per kWh (typical grid emission factor)

    print(f"\n{Fore.CYAN}⚡ Comparing routing algorithms across {len(od_pairs)} O-D pairs...{Style.RESET_ALL}")
    
    for i, (origin, destination) in enumerate(od_pairs):
        print(f"\n{Fore.YELLOW}Processing O-D Pair {i+1}/{len(od_pairs)}: {origin} → {destination}{Style.RESET_ALL}")
        
        try:
            # Set 30-second timeout for CRPTC algorithm (prevent hanging on large networks)
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(30)

            # Run all five routing algorithms
            h_path, h_energy, h_metrics = heuristic_routing_algorithm(graph, origin, destination, vehicle_params)
            e_path, e_energy, e_metrics = eco_routing_algorithm(graph, origin, destination, vehicle_params)
            s_path, s_energy, s_metrics = shortest_path_distance(graph, origin, destination, vehicle_params)
            f_path, f_energy, f_metrics = fastest_path_time(graph, origin, destination, vehicle_params)
            c_path, c_energy, c_metrics = crptc_algorithm(graph, origin, destination, vehicle_params)

            def calculate_green_zone_coverage(path):
                """
                Calculate percentage of route traversing green zones.
                
                Green zone coverage is computed as the ratio of green-zone road length
                to total route length, expressed as a percentage.
                
                Parameters:
                    path (list): Sequence of node IDs
                    
                Returns:
                    float: Green zone coverage percentage (0-100)
                """
                green_zone_length = sum(graph[path[i]][path[i+1]]['length'] for i in range(len(path)-1)
                                        if graph[path[i]][path[i+1]].get('in_green_zone', False))
                total_length = sum(graph[path[i]][path[i+1]]['length'] for i in range(len(path)-1))
                return (green_zone_length / total_length * 100) if total_length > 0 else 0

            # Calculate green zone coverage for all algorithms
            h_green_coverage = calculate_green_zone_coverage(h_path)
            e_green_coverage = calculate_green_zone_coverage(e_path)
            s_green_coverage = calculate_green_zone_coverage(s_path)
            f_green_coverage = calculate_green_zone_coverage(f_path)
            c_green_coverage = calculate_green_zone_coverage(c_path)

            # Energy efficiency: kWh per kilometer
            h_energy_per_km = (h_energy * 1000 / h_metrics["distance"]) if h_metrics["distance"] > 0 else float('inf')
            e_energy_per_km = (e_energy * 1000 / e_metrics["distance"]) if e_metrics["distance"] > 0 else float('inf')
            s_energy_per_km = (s_energy * 1000 / s_metrics["distance"]) if s_metrics["distance"] > 0 else float('inf')
            f_energy_per_km = (f_energy * 1000 / f_metrics["distance"]) if f_metrics["distance"] > 0 else float('inf')
            c_energy_per_km = (c_energy * 1000 / c_metrics["distance"]) if c_metrics["distance"] > 0 else float('inf')

            # CO₂ emissions: only count positive energy consumption (grid electricity)
            h_co2 = max(0, h_energy) * co2_per_kwh
            e_co2 = max(0, e_energy) * co2_per_kwh
            s_co2 = max(0, s_energy) * co2_per_kwh
            f_co2 = max(0, f_energy) * co2_per_kwh
            c_co2 = max(0, c_energy) * co2_per_kwh

            # Battery utilization: percentage of initial charge consumed
            h_battery_util = (h_energy / vehicle_params["initial_charge"] * 100) if h_energy > 0 else 0
            e_battery_util = (e_energy / vehicle_params["initial_charge"] * 100) if e_energy > 0 else 0
            s_battery_util = (s_energy / vehicle_params["initial_charge"] * 100) if s_energy > 0 else 0
            f_battery_util = (f_energy / vehicle_params["initial_charge"] * 100) if f_energy > 0 else 0
            c_battery_util = (c_energy / vehicle_params["initial_charge"] * 100) if c_energy > 0 else 0

            # Range safety: remaining charge relative to minimum acceptable level
            h_range_safety = (h_metrics["remaining_charge"] / vehicle_params["min_charge"] * 100) if vehicle_params["min_charge"] > 0 else 100
            e_range_safety = (e_metrics["remaining_charge"] / vehicle_params["min_charge"] * 100) if vehicle_params["min_charge"] > 0 else 100
            s_range_safety = (s_metrics["remaining_charge"] / vehicle_params["min_charge"] * 100) if vehicle_params["min_charge"] > 0 else 100
            f_range_safety = (f_metrics["remaining_charge"] / vehicle_params["min_charge"] * 100) if vehicle_params["min_charge"] > 0 else 100
            c_range_safety = (c_metrics["remaining_charge"] / vehicle_params["min_charge"] * 100) if vehicle_params["min_charge"] > 0 else 100

            # Time efficiency: seconds per kilometer
            h_time_per_km = (h_metrics["time"] / (h_metrics["distance"]/1000)) if h_metrics["distance"] > 0 else float('inf')
            e_time_per_km = (e_metrics["time"] / (e_metrics["distance"]/1000)) if e_metrics["distance"] > 0 else float('inf')
            s_time_per_km = (s_metrics["time"] / (s_metrics["distance"]/1000)) if s_metrics["distance"] > 0 else float('inf')
            f_time_per_km = (f_metrics["time"] / (f_metrics["distance"]/1000)) if f_metrics["distance"] > 0 else float('inf')
            c_time_per_km = (c_metrics["time"] / (c_metrics["distance"]/1000)) if c_metrics["distance"] > 0 else float('inf')

            # Cost efficiency: dollars per kilometer
            h_cost_per_km = (h_metrics["total_cost"] / (h_metrics["distance"]/1000)) if h_metrics["distance"] > 0 else float('inf')
            e_cost_per_km = (e_metrics["total_cost"] / (e_metrics["distance"]/1000)) if e_metrics["distance"] > 0 else float('inf')
            s_cost_per_km = (s_metrics["total_cost"] / (s_metrics["distance"]/1000)) if s_metrics["distance"] > 0 else float('inf')
            f_cost_per_km = (f_metrics["total_cost"] / (f_metrics["distance"]/1000)) if f_metrics["distance"] > 0 else float('inf')
            c_cost_per_km = (c_metrics["total_cost"] / (c_metrics["distance"]/1000)) if c_metrics["distance"] > 0 else float('inf')

            # Comparative metrics: Heuristic vs Eco-Routing
            h_vs_e_energy = ((h_energy - e_energy) / e_energy * 100) if e_energy > 0 else 0
            h_vs_e_green_pp = h_green_coverage - e_green_coverage  # Percentage point difference
            h_vs_e_time = ((h_metrics["time"] - e_metrics["time"]) / e_metrics["time"] * 100) if e_metrics["time"] > 0 else 0
            h_vs_e_cost = ((h_metrics["total_cost"] - e_metrics["total_cost"]) / e_metrics["total_cost"] * 100) if e_metrics["total_cost"] > 0 else 0

            # Comparative metrics: Heuristic vs CRPTC
            h_vs_c_energy = ((h_energy - c_energy) / c_energy * 100) if c_energy > 0 else 0
            h_vs_c_green_pp = h_green_coverage - c_green_coverage
            h_vs_c_time = ((h_metrics["time"] - c_metrics["time"]) / c_metrics["time"] * 100) if c_metrics["time"] > 0 else 0
            h_vs_c_cost = ((h_metrics["total_cost"] - c_metrics["total_cost"]) / c_metrics["total_cost"] * 100) if c_metrics["total_cost"] > 0 else 0

            # V2G efficiency: incentive revenue per kWh of battery capacity
            h_v2g_efficiency = h_metrics["v2g_incentives"] / vehicle_params["battery_capacity"] if vehicle_params["battery_capacity"] > 0 else 0

            # Package all results for this OD pair
            pair_results = {
                'od_pair': f"{origin}-{destination}",
                'distance': h_metrics["distance"],
                'h_energy': h_energy,
                'e_energy': e_energy,
                'h_time': h_metrics["time"],
                'e_time': e_metrics["time"],
                'h_cost': h_metrics["total_cost"],
                'e_cost': e_metrics["total_cost"],
                'h_green_coverage': h_green_coverage,
                'e_green_coverage': e_green_coverage,
                'h_vs_e_energy_pct': h_vs_e_energy,
                'h_vs_e_green_pp': h_vs_e_green_pp,
                'h_vs_e_time_pct': h_vs_e_time,
                'h_vs_e_cost_pct': h_vs_e_cost,
                'h_vs_c_energy_pct': h_vs_c_energy,
                'h_vs_c_green_pp': h_vs_c_green_pp,
                'h_vs_c_time_pct': h_vs_c_time,
                'h_vs_c_cost_pct': h_vs_c_cost,
                'h_v2g_incentives': h_metrics["v2g_incentives"],
                'h_v2g_efficiency': h_v2g_efficiency,
                'h_green_penalties': h_metrics["green_zone_penalties"],
                'h_success': len(h_path) > 0 and "warning" not in h_metrics,
                'e_success': len(e_path) > 0 and "warning" not in e_metrics,
                's_success': len(s_path) > 0 and "warning" not in s_metrics,
                'f_success': len(f_path) > 0 and "warning" not in f_metrics,
                'c_success': len(c_path) > 0 and "warning" not in c_metrics
            }
            results.append(pair_results)
            
            # Console output: key metrics for this OD pair
            print(f"  {Fore.GREEN}Energy (kWh):{Style.RESET_ALL} Heuristic: {h_energy:.2f}, Eco: {e_energy:.2f}, CRPTC: {c_energy:.2f}")
            print(f"  {Fore.GREEN}Time (s):{Style.RESET_ALL} Heuristic: {h_metrics['time']:.2f}, Eco: {e_metrics['time']:.2f}, CRPTC: {c_metrics['time']:.2f}")

        except TimeoutException:
            # CRPTC timeout: skip this OD pair
            print(f"{Fore.Red}CRPTC algorithm timed out for OD pair {i+1}. Skipping...{Style.RESET_ALL}")
        except Exception as e:
            # General error: print and continue
            print(f"{Fore.RED}Error processing O-D pair {i+1}: {str(e)}{Style.RESET_ALL}")
        finally:
            # Cancel alarm signal regardless of outcome
            signal.alarm(0)

    # Convert results to DataFrame for analysis
    results_df = pd.DataFrame(results)
    
    # Compute aggregate statistics across all successful OD pairs
    agg_stats = {}
    successful_results = results_df[(results_df['h_success']) & (results_df['e_success'])]
    
    if len(successful_results) > 0:
        # Average energy consumption
        agg_stats['avg_h_energy'] = successful_results['h_energy'].mean()
        agg_stats['avg_e_energy'] = successful_results['e_energy'].mean()
        agg_stats['avg_c_energy'] = successful_results['c_energy'].mean()

        # Average travel time
        agg_stats['avg_h_time'] = successful_results['h_time'].mean()
        agg_stats['avg_e_time'] = successful_results['e_time'].mean()
        agg_stats['avg_c_time'] = successful_results['c_time'].mean()

        # Average green zone coverage
        agg_stats['avg_h_green_coverage'] = successful_results['h_green_coverage'].mean()
        agg_stats['avg_e_green_coverage'] = successful_results['e_green_coverage'].mean()  # Typically 0 for eco-routing
        agg_stats['avg_c_green_coverage'] = successful_results['c_green_coverage'].mean()  # Typically 0 for CRPTC

        # Average V2G incentives
        agg_stats['avg_h_v2g_incentives'] = successful_results['h_v2g_incentives'].mean()
        agg_stats['avg_e_v2g_incentives'] = successful_results['e_v2g_incentives'].mean()  # 0 for eco-routing
        agg_stats['avg_c_v2g_incentives'] = successful_results['c_v2g_incentives'].mean()  # 0 for CRPTC

        # Average percentage improvements
        agg_stats['avg_h_vs_e_energy'] = successful_results['h_vs_e_energy_pct'].mean()
        agg_stats['avg_h_vs_c_energy'] = successful_results['h_vs_c_energy_pct'].mean()
        
        agg_stats['avg_h_vs_e_green_pp'] = successful_results['h_vs_e_green_pp'].mean()
        agg_stats['avg_h_vs_c_green_pp'] = successful_results['h_vs_c_green_pp'].mean()
        
        agg_stats['avg_h_vs_e_time'] = successful_results['h_vs_e_time_pct'].mean()
        agg_stats['avg_h_vs_c_time'] = successful_results['h_vs_c_time_pct'].mean()
        
        agg_stats['avg_h_vs_e_cost'] = successful_results['h_vs_e_cost_pct'].mean()
        agg_stats['avg_h_vs_c_cost'] = successful_results['h_vs_c_cost_pct'].mean()
        
    return results_df, agg_stats


In [ ]:
def prepare_experiment(num_nodes=2000, num_edges=4000, num_charging_stations=80, num_od_pairs=500, min_distance=1000):
    """
    Prepare the complete experimental environment for routing algorithm evaluation.
    
    This function sets up all necessary components for a controlled routing experiment:
    1. Generates synthetic urban road network with realistic attributes
    2. Deploys charging infrastructure with V2G capabilities
    3. Designates green zones for eco-routing
    4. Selects diverse origin-destination pairs for testing
    
    The generated environment provides a reproducible testbed for comparing routing
    algorithms under consistent conditions.

    Parameters:
        num_nodes (int, optional): Number of intersections in the road network.
                                   Default: 2000 (creates ~45×45 grid).
        num_edges (int, optional): Number of road segments including shortcuts.
                                  Default: 4000 (2× nodes for good connectivity).
        num_charging_stations (int, optional): Number of charging stations to deploy.
                                              Default: 80 (4% of 2000 nodes).
        num_od_pairs (int, optional): Number of test OD pairs to generate.
                                     Default: 500 (sufficient for statistical significance).
        min_distance (int, optional): Minimum distance between OD pairs in meters.
                                     Default: 1000 (1 km - short urban trips).

    Returns:
        tuple: (road_network, od_pairs)
            - road_network (nx.Graph): Complete network with all attributes
            - od_pairs (list): List of (origin, destination) tuples for testing

    Network Statistics Printed:
        - Total nodes and edges
        - Number of charging stations (with V2G enabled)
        - Number of green zone edges (30% typically)
        - Number of valid OD pairs selected

    Notes:
        - Seed 42 used for reproducibility (can be modified in generate_road_network)
        - City extent: 30 km × 30 km
        - Charging station density: typically 4-5% of nodes
        - Green zone coverage: 30% of edges
        - V2G incentive rates adjusted to 1.2× charging cost
        
    Recommended Configurations:
        - Quick test: num_nodes=400, num_edges=800, num_od_pairs=50
        - Standard test: num_nodes=2000, num_edges=4000, num_od_pairs=500
        - Large-scale: num_nodes=10000, num_edges=20000, num_od_pairs=2000
    """
    # Set random seeds for reproducibility across experiments
    np.random.seed(42)
    random.seed(42)

    # Generate synthetic urban road network
    print(f"\n{Fore.YELLOW}Generating road network...{Style.RESET_ALL}")
    road_network = generate_road_network(
        num_nodes=num_nodes, 
        num_edges=num_edges, 
        num_charging_stations=num_charging_stations
    )

    # Configure V2G incentive rates at all charging stations
    # Rate set to 1.2× base electricity cost (20% premium for grid services)
    for node in road_network.nodes():
        if road_network.nodes[node].get('charging_station', False):
            road_network.nodes[node]['incentive_rate'] = 1.2 * VEHICLE_PARAMETERS["cost_per_kwh"]

    # Print network statistics
    print(f"Road network: {road_network.number_of_nodes()} nodes, {road_network.number_of_edges()} edges")
    
    num_charging = len([n for n in road_network.nodes() if road_network.nodes[n].get('charging_station', False)])
    print(f"Charging stations: {num_charging}")
    
    num_green_edges = sum(1 for u, v in road_network.edges() if road_network[u][v].get('in_green_zone', False))
    print(f"Green zone edges: {num_green_edges}")

    # Generate origin-destination test pairs
    print(f"\n{Fore.YELLOW}Selecting origin-destination pairs...{Style.RESET_ALL}")
    od_pairs = select_random_od_pairs(road_network, num_pairs=num_od_pairs, min_distance=min_distance)
    print(f"Selected {len(od_pairs)} OD pairs.")

    return road_network, od_pairs


def run_minimal_experiment(road_network, od_pairs):
    """
    Execute routing experiment comparing all algorithms on given OD pairs.
    
    This function runs the complete experimental pipeline:
    1. Initializes vehicle parameters with fixed battery state
    2. Processes each OD pair through all five routing algorithms
    3. Computes comprehensive performance metrics
    4. Aggregates results for statistical analysis
    
    Algorithms Evaluated:
        - Heuristic: Green zone + V2G optimization (primary contribution)
        - Eco-Routing: Energy minimization baseline
        - Shortest Path: Distance minimization baseline
        - Fastest Path: Time minimization baseline
        - CRPTC: Cost optimization baseline
    
    Metrics Computed (per OD pair):
        - Energy Consumed (kWh): Total battery energy used
        - Time Taken (seconds): Travel time including charging delays
        - Total Cost ($): Energy + charging + penalties - V2G revenue
        - Green Zone Coverage (%): Percentage of route in eco-friendly zones
        - V2G Incentives ($): Revenue from Vehicle-to-Grid discharge

    Parameters:
        road_network (nx.Graph): Road network from prepare_experiment().
        od_pairs (list): List of (origin, destination) tuples to evaluate.

    Returns:
        tuple: (road_network, comparison_results)
            - road_network (nx.Graph): Same network (for reference)
            - comparison_results (list): List of dictionaries with per-OD-pair metrics

    Output Format:
        Console output includes:
        - Experiment timestamp and user info
        - Progress for each OD pair
        - Algorithm performance summary per pair
        - Color-coded status messages (yellow=progress, green=success, red=error)

    Usage Example:
        ```python
        # Setup
        network, od_pairs = prepare_experiment(num_nodes=1000, num_od_pairs=100)
        
        # Run experiment
        network, results = run_minimal_experiment(network, od_pairs)
        
        # Analyze results
        df = pd.DataFrame(results)
        print(df[['h_energy', 'e_energy', 'h_green_coverage']].describe())
        ```

    Notes:
        - Fixed initial charge (80 kWh) ensures fair comparison across all OD pairs
        - Each algorithm runs independently (failures don't affect others)
        - Results suitable for statistical analysis and visualization
        - Can export results to CSV for publication figures
    """
    print(f"\n{Fore.CYAN}Running Minimal EV Routing Experiment - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}{Style.RESET_ALL}")
    print(f"User: kripa-sindhu-007")

    # Use fixed initial charge for consistent comparison across experiments
    vehicle_params = VEHICLE_PARAMETERS.copy()
    vehicle_params["initial_charge"] = 80  # kWh (80% of 100 kWh battery)

    def compute_green_zone_coverage(path, graph):
        """
        Compute percentage of path length traversing green zones.
        
        Green zone coverage indicates how much of the route benefits from
        environmentally-friendly infrastructure (renewable charging, low emissions,
        optimized traffic management).
        
        Parameters:
            path (list): Sequence of node IDs forming the route
            graph (nx.Graph): Road network with green zone attributes
            
        Returns:
            float: Green zone coverage as percentage (0-100), rounded to 2 decimals
        """
        total_length = 0.0
        green_length = 0.0
        
        for i in range(len(path) - 1):
            u, v = path[i], path[i+1]
            edge_length = graph[u][v].get('length', 0.0)
            total_length += edge_length
            
            if graph[u][v].get('in_green_zone', False):
                green_length += edge_length
        
        # Return percentage with 2 decimal precision
        return round((green_length / total_length * 100), 2) if total_length > 0 else 0.0

    comparison_results = []
    total_pairs = len(od_pairs)

    # Process each OD pair sequentially
    for i, (origin, destination) in enumerate(od_pairs):
        print(f"\n{Fore.YELLOW}Processing OD Pair {i+1}/{total_pairs}: {origin} → {destination}{Style.RESET_ALL}")
        
        try:
            # Run all five routing algorithms
            h_path, h_energy, h_metrics = heuristic_routing_algorithm(road_network, origin, destination, vehicle_params)
            e_path, e_energy, e_metrics = eco_routing_algorithm(road_network, origin, destination, vehicle_params)
            s_path, s_energy, s_metrics = shortest_path_distance(road_network, origin, destination, vehicle_params)
            f_path, f_energy, f_metrics = fastest_path_time(road_network, origin, destination, vehicle_params)
            c_path, c_energy, c_metrics = crptc_algorithm(road_network, origin, destination, vehicle_params)

            # Calculate green zone coverage for each algorithm
            h_gz = compute_green_zone_coverage(h_path, road_network)
            e_gz = 0.0  # Eco-routing doesn't prioritize green zones (pure energy minimization)
            s_gz = 0.0  # Shortest path doesn't consider green zones
            f_gz = 0.0  # Fastest path doesn't consider green zones
            c_gz = 0.0  # CRPTC doesn't explicitly optimize for green zones

            # Package results for this OD pair
            result = {
                "origin": origin,
                "destination": destination,
                
                # Heuristic algorithm results
                "h_energy": h_energy,
                "h_time": h_metrics.get("time", np.nan),
                "h_cost": h_metrics.get("total_cost", np.nan),
                "h_green_coverage": h_gz,
                "h_v2g_incentives": h_metrics.get("v2g_incentives", 0),

                # Eco-routing algorithm results
                "e_energy": e_energy,
                "e_time": e_metrics.get("time", np.nan),
                "e_cost": e_metrics.get("total_cost", np.nan),
                "e_green_coverage": e_gz,
                "e_v2g_incentives": e_metrics.get("v2g_incentives", 0),

                # CRPTC algorithm results (cost optimization baseline)
                "c_energy": c_energy,
                "c_time": c_metrics.get("time", np.nan),
                "c_cost": c_metrics.get("total_cost", np.nan),
                "c_green_coverage": c_gz,
            }
            comparison_results.append(result)

            # Print summary for this OD pair
            print(f"  Heuristic: Energy = {h_energy:.2f} kWh, Time = {h_metrics.get('time',0):.2f} s, Cost = ${h_metrics.get('total_cost',0):.2f}, Green = {h_gz:.1f}%, V2G = ${h_metrics.get('v2g_incentives',0):.2f}")
            print(f"  Eco-Routing: Energy = {e_energy:.2f} kWh, Time = {e_metrics.get('time',0):.2f} s, Cost = ${e_metrics.get('total_cost',0):.2f}, Green = {e_gz:.1f}%, V2G = ${e_metrics.get('v2g_incentives',0):.2f}")
            print(f"  Shortest Path: Energy = {s_energy:.2f} kWh, Time = {s_metrics.get('time',0):.2f} s, Cost = ${s_metrics.get('total_cost',0):.2f}, Green = {s_gz:.1f}%, V2G = ${s_metrics.get('v2g_incentives',0):.2f}")
            print(f"  Fastest Path: Energy = {f_energy:.2f} kWh, Time = {f_metrics.get('time',0):.2f} s, Cost = ${f_metrics.get('total_cost',0):.2f}, Green = {f_gz:.1f}%, V2G = ${f_metrics.get('v2g_incentives',0):.2f}")
            print(f"  CRPTC: Energy = {c_energy:.2f} kWh, Time = {c_metrics.get('time',0):.2f} s, Cost = ${c_metrics.get('total_cost',0):.2f}, Green = {c_gz:.1f}%, V2G = ${c_metrics.get('v2g_incentives',0):.2f}")
            
        except Exception as ex:
            # Log error and continue with next OD pair
            print(f"{Fore.RED}Error processing OD pair {i+1}: {ex}{Style.RESET_ALL}")

    print(f"\n{Fore.CYAN}===== Experiment Completed ====={Style.RESET_ALL}")
    return road_network, comparison_results


In [ ]:
# Prepare the experiment environment.
road_network, od_pairs = prepare_experiment(num_nodes=4000, num_edges=8000, num_charging_stations=160, num_od_pairs=10, min_distance=5000)

# Run the simulation using the generated network and OD pairs.
road_network, comp_results = run_minimal_experiment(road_network, od_pairs)

# Convert results to a DataFrame and display summary.
# comp_df = pd.DataFrame(comp_results)
# print("\n===== Overall Comparison Metrics (per OD Pair) =====")
# print(tabulate(comp_df.head(10), headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
def visualize_longest_OD_improved(road_network, od_pairs):
    """
    Visualize the longest OD pair (based on Eco-Routing distance) on a 2D map, highlighting:
      - Green zone edges (in a brighter green with increased line width).
      - Charging stations (gold diamonds with black borders).
      - Four routing paths (Heuristic, Eco-Routing, Shortest Path, Fastest Path) in distinct colors & linestyles.
      - Origin (lime) and destination (crimson).

    The entire urban network is drawn in a light gray color with a low alpha to provide context.
    """

    import matplotlib.pyplot as plt
    import networkx as nx

    # Compute distance of a path
    def compute_path_distance(path, graph):
        return sum(graph[u][v]['length'] for u, v in zip(path[:-1], path[1:]))

    # 1. Find the longest OD pair
    max_distance = 0
    longest_pair = None
    for (origin, destination) in od_pairs:
        try:
            eco_path, _, _ = eco_routing_algorithm(road_network, origin, destination, VEHICLE_PARAMETERS)
            d = compute_path_distance(eco_path, road_network)
            if d > max_distance:
                max_distance = d
                longest_pair = (origin, destination)
        except:
            continue

    if longest_pair is None:
        print("No valid OD pair found for visualization.")
        return

    origin, destination = longest_pair
    print(f"Longest OD Pair: {origin} → {destination}, Distance: {max_distance:.2f} m")

    # 2. Run all four algorithms on the longest OD pair
    h_path, _, _ = heuristic_routing_algorithm(road_network, origin, destination, VEHICLE_PARAMETERS)
    e_path, _, _ = eco_routing_algorithm(road_network, origin, destination, VEHICLE_PARAMETERS)
    s_path, _, _ = shortest_path_distance(road_network, origin, destination, VEHICLE_PARAMETERS)
    f_path, _, _ = fastest_path_time(road_network, origin, destination, VEHICLE_PARAMETERS)

    # 3. Prepare data structures for plotting
    paths_dict = {
        "Heuristic": h_path,
        "Eco-Routing": e_path,
        "Shortest Path": s_path,
        "Fastest Path": f_path
    }

    route_colors = {
    "Heuristic": "#1f77b4",    # Classic blue
    "Eco-Routing": "#8c564b",  # Warm Brown
    "Shortest Path": "#ff7f0e",# Bold orange
    "Fastest Path": "#9467bd"  # Cool purple
    }

    route_styles = {
    "Heuristic": "solid",
    "Eco-Routing": "dashed",
    "Shortest Path": "dashdot",
    "Fastest Path": "dotted"
    }

    # Node positions
    pos = nx.get_node_attributes(road_network, 'pos')

    # 4. Create figure
    plt.figure(figsize=(24, 16))  # Larger figure for clarity
    ax = plt.gca()

    # Draw entire network edges in a light gray color, thin lines
    nx.draw_networkx_edges(
        road_network, pos,
        alpha=0.15, width=0.7, edge_color="black"
    )

    # Highlight green zone edges
    green_edges = [(u, v) for (u, v) in road_network.edges() if road_network[u][v].get('in_green_zone', False)]
    if green_edges:
        nx.draw_networkx_edges(
            road_network, pos, edgelist=green_edges,
            edge_color="green", width=0.7, alpha=0.8,
            label="Green Zone Edge"
        )

    # Draw nodes in a subtle color, small size
    nx.draw_networkx_nodes(
        road_network, pos,
        node_size=10, node_color="black"
    )

    # Charging stations (gold diamond, black border)
    charging_nodes = [n for n in road_network.nodes() if road_network.nodes[n].get('charging_station', False)]
    if charging_nodes:
        nx.draw_networkx_nodes(
            road_network, pos, nodelist=charging_nodes,
            node_color="gold", node_shape="D", node_size=100,
            edgecolors="black", linewidths=0.8,
            label="Charging Station"
        )

    # Overlay each route with distinct color & style
    for label, path in paths_dict.items():
        path_edges = list(zip(path[:-1], path[1:]))
        nx.draw_networkx_edges(
            road_network, pos, edgelist=path_edges,
            width=3, edge_color=route_colors[label],
            style=route_styles[label],
            label=label
        )

    # Mark origin and destination
    nx.draw_networkx_nodes(
        road_network, pos, nodelist=[origin],
        node_color="lime", node_size=300, edgecolors="black",
        linewidths=1.0, label="Origin"
    )
    nx.draw_networkx_nodes(
        road_network, pos, nodelist=[destination],
        node_color="crimson", node_size=300, edgecolors="black",
        linewidths=1.0, label="Destination"
    )

    # 5. Legend & title
    handles, labels = ax.get_legend_handles_labels()
    unique_labels = []
    unique_handles = []
    for h, l in zip(handles, labels):
        if l not in unique_labels:
            unique_labels.append(l)
            unique_handles.append(h)

    ax.legend(unique_handles, unique_labels, fontsize=14, loc="best", frameon=True)
    ax.set_title(
        f"Urban Network & Routing Paths for Longest OD Pair: {origin} → {destination}\nDistance: {max_distance:.2f} m",
        fontsize=22, fontweight="bold", pad=20
    )

    plt.axis("off")
    plt.tight_layout()
    plt.show()

# Run the Visualization
visualize_longest_OD_improved(road_network, od_pairs)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Convert experiment results (list of dictionaries) into a DataFrame.
comp_df = pd.DataFrame(comp_results)

# Define the algorithms and their prefixes.
algo_keys = {'h': 'Heuristic', 'e': 'Eco-Routing', 's': 'Shortest Path', 'f': 'Fastest Path' , 'c':'CRPTC'}
algos = list(algo_keys.keys())

# Define the metrics to display in the absolute summary table.
abs_metrics = {
    'energy': 'Energy Consumed (kWh)',
    'time': 'Time Taken (s)',
    'cost': 'Total Cost ($)',
    'green_coverage': 'Green Zone Coverage (%)',
    'v2g_incentives': 'V2G Incentives ($)'
}
# Optionally include distance if available.
if 'h_distance' in comp_df.columns:
    abs_metrics['distance'] = 'Distance Travelled (m)'

# -----------------------------------------------------------------------------
# Part 1: Absolute Summary Table (mean ± std) for all algorithms
abs_summary = []
for metric_key, metric_label in abs_metrics.items():
    for algo in algos:
        col = f"{algo}_{metric_key}"
        if col in comp_df.columns:
            mean_val = comp_df[col].mean()
            std_val = comp_df[col].std()
            abs_summary.append({
                "Metric": metric_label,
                "Algorithm": algo_keys[algo],
                "Average": mean_val,
                "Std Dev": std_val
            })

abs_summary_df = pd.DataFrame(abs_summary)
abs_table = abs_summary_df.pivot(index="Metric", columns="Algorithm", values=["Average", "Std Dev"]).round(2)
print("===== Absolute Metrics (per OD Pair) =====")
print(tabulate(abs_table, headers="keys", tablefmt="pretty"))

# -----------------------------------------------------------------------------
# Part 2: Percentage Improvement (Heuristic vs Eco-Routing)
rel_metrics = {}
for metric_key, metric_label in abs_metrics.items():
    if metric_key in ['v2g_incentives', 'green_coverage']:
        continue  # Skip these metrics.
    if f"e_{metric_key}" in comp_df.columns and f"h_{metric_key}" in comp_df.columns:
        # For metrics where lower is better (energy, time, cost, distance), improvement = ((Eco - Heuristic)/Eco)*100.
        improvement1 = ((comp_df[f"e_{metric_key}"] - comp_df[f"h_{metric_key}"]) / comp_df[f"e_{metric_key}"].replace(0, 1)) * 100
        rel_metrics[metric_key] = improvement1

# Build a summary table for percentage improvements.
rel_summary = []
for metric_key, metric_label in abs_metrics.items():
    if metric_key in rel_metrics:
        mean_impr = rel_metrics[metric_key].mean()
        std_impr = rel_metrics[metric_key].std()
        rel_summary.append({
            "Metric": metric_label + " Improvement (%)",
            "Heuristic vs Eco": f"{mean_impr:.2f} ± {std_impr:.2f}"
        })
rel_summary_df = pd.DataFrame(rel_summary)
print("\n===== Percentage Improvement (Heuristic vs Eco-Routing) =====")
print(tabulate(rel_summary_df, headers="keys", tablefmt="pretty"))

# -----------------------------------------------------------------------------
# Part 3: Graphs

# (A) Subplots for Absolute Metrics (one subplot per metric)
num_metrics = len(abs_metrics)
fig, axes = plt.subplots(1, num_metrics, figsize=(6*num_metrics, 8))
base_colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']
for i, (metric_key, metric_label) in enumerate(abs_metrics.items()):
    ax = axes[i] if num_metrics > 1 else axes
    means = []
    stds = []
    labels = []
    for algo in algos:
        col = f"{algo}_{metric_key}"
        if col in comp_df.columns:
            means.append(comp_df[col].mean())
            stds.append(comp_df[col].std())
            labels.append(algo_keys[algo])
    x = np.arange(len(labels))
    c_slice = base_colors[:len(labels)]
    ax.bar(x, means, yerr=stds, capsize=5, color=c_slice)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45)
    ax.set_title(metric_label, fontsize=12, fontweight='bold')
    ax.set_ylabel(metric_label)
plt.suptitle("Absolute Metrics Comparison Across Algorithms", fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# (B) Single Bar Chart for Percentage Improvement (Heuristic vs Eco-Routing)
rel_summary_plot = []
for metric_key, metric_label in abs_metrics.items():
    if metric_key in rel_metrics:
        mean_impr = rel_metrics[metric_key].mean()
        std_impr = rel_metrics[metric_key].std()
        rel_summary_plot.append({
            "Metric": metric_label,
            "Average Improvement (%)": mean_impr,
            "Std Dev": std_impr
        })
rel_summary_plot_df = pd.DataFrame(rel_summary_plot)

plt.figure(figsize=(8,6))
ax = plt.gca()
color_val = plt.get_cmap("coolwarm")(0.6)
bars = ax.bar(rel_summary_plot_df["Metric"],
              rel_summary_plot_df["Average Improvement (%)"],
              yerr=rel_summary_plot_df["Std Dev"],
              capsize=5, color=color_val)
ax.set_title("Average Percentage Improvement (Heuristic vs Eco-Routing)", fontsize=16, fontweight='bold')
ax.set_ylabel("Improvement (%)")
ax.set_xlabel("")
plt.xticks(rotation=45, ha='right')

# Add numeric labels on top of each bar.
for idx, row in rel_summary_plot_df.iterrows():
    val = row["Average Improvement (%)"]
    ax.text(idx, val + (1.5 if val >= 0 else -1.5),
            f"{val:.1f}%", ha='center',
            va='bottom' if val >= 0 else 'top',
            fontsize=11, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Assume comp_results is provided as a list of dictionaries.
comp_df = pd.DataFrame(comp_results)

# Define only Heuristic and Eco-Routing algorithms.
algo_keys = {'h': 'Heuristic', 'e': 'Eco-Routing'}
algos = list(algo_keys.keys())

# Define the metrics to display in the absolute summary table.
abs_metrics = {
    'energy': 'Energy Consumed (kWh)',
    'time': 'Time Taken (s)',
    'cost': 'Total Cost ($)',
    'green_coverage': 'Green Zone Coverage (%)',
    'v2g_incentives': 'V2G Incentives ($)'
}
# Optionally include distance if available.
if 'h_distance' in comp_df.columns:
    abs_metrics['distance'] = 'Distance Travelled (m)'

# -------------------------------------------------------------------
# Part 1: Absolute Summary Table (mean ± std) for Heuristic and Eco-Routing.
abs_summary = []
for metric_key, metric_label in abs_metrics.items():
    for algo in algos:
        col = f"{algo}_{metric_key}"
        if col in comp_df.columns:
            mean_val = comp_df[col].mean()
            std_val = comp_df[col].std()
            abs_summary.append({
                "Metric": metric_label,
                "Algorithm": algo_keys[algo],
                "Average": mean_val,
                "Std Dev": std_val
            })

abs_summary_df = pd.DataFrame(abs_summary)
abs_table = abs_summary_df.pivot(index="Metric", columns="Algorithm", values=["Average", "Std Dev"]).round(2)
print("===== Absolute Metrics (per OD Pair) =====")
print(tabulate(abs_table, headers="keys", tablefmt="pretty"))

# -------------------------------------------------------------------
# Part 2: Percentage Improvement (Heuristic vs Eco-Routing)
# For metrics where lower is better, improvement = ((Eco - Heuristic)/Eco)*100.
rel_metrics = {}
for metric_key, metric_label in abs_metrics.items():
    # Skip metrics where percentage improvement is not meaningful.
    if metric_key in ['v2g_incentives', 'green_coverage']:
        continue
    col_eco = f"e_{metric_key}"
    col_heu = f"h_{metric_key}"
    if col_eco in comp_df.columns and col_heu in comp_df.columns:
        improvement = ((comp_df[col_eco] - comp_df[col_heu]) / comp_df[col_eco].replace(0, 1)) * 100
        rel_metrics[metric_key] = improvement

# Build a summary table for percentage improvements.
rel_summary = []
for metric_key, metric_label in abs_metrics.items():
    if metric_key in rel_metrics:
        mean_impr = rel_metrics[metric_key].mean()
        std_impr = rel_metrics[metric_key].std()
        rel_summary.append({
            "Metric": metric_label + " Improvement (%)",
            "Heuristic vs Eco": f"{mean_impr:.2f} ± {std_impr:.2f}"
        })
rel_summary_df = pd.DataFrame(rel_summary)
print("\n===== Percentage Improvement (Heuristic vs Eco-Routing) =====")
print(tabulate(rel_summary_df, headers="keys", tablefmt="pretty"))

# -------------------------------------------------------------------
# Part 3: Graphs

# (A) Subplots for Absolute Metrics (one subplot per metric)
num_metrics = len(abs_metrics)
fig, axes = plt.subplots(1, num_metrics, figsize=(6*num_metrics, 8))
base_colors = ['#3498db', '#2ecc71']  # Colors for Heuristic and Eco-Routing
for i, (metric_key, metric_label) in enumerate(abs_metrics.items()):
    ax = axes[i] if num_metrics > 1 else axes
    means = []
    stds = []
    labels = []
    for algo in algos:
        col = f"{algo}_{metric_key}"
        if col in comp_df.columns:
            means.append(comp_df[col].mean())
            stds.append(comp_df[col].std())
            labels.append(algo_keys[algo])
    x = np.arange(len(labels))
    ax.bar(x, means, yerr=stds, capsize=5, color=base_colors[:len(labels)])
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45)
    ax.set_title(metric_label, fontsize=12, fontweight='bold')
    ax.set_ylabel(metric_label)
plt.suptitle("Absolute Metrics Comparison: Heuristic vs Eco-Routing", fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# (B) Single Bar Chart for Percentage Improvement (Heuristic vs Eco-Routing)
rel_summary_plot = []
for metric_key, metric_label in abs_metrics.items():
    if metric_key in rel_metrics:
        mean_impr = rel_metrics[metric_key].mean()
        std_impr = rel_metrics[metric_key].std()
        rel_summary_plot.append({
            "Metric": metric_label,
            "Average Improvement (%)": mean_impr,
            "Std Dev": std_impr
        })
rel_summary_plot_df = pd.DataFrame(rel_summary_plot)

plt.figure(figsize=(8,6))
ax = plt.gca()
color_val = plt.get_cmap("coolwarm")(0.6)
bars = ax.bar(rel_summary_plot_df["Metric"],
              rel_summary_plot_df["Average Improvement (%)"],
              yerr=rel_summary_plot_df["Std Dev"],
              capsize=5, color=color_val)
ax.set_title("Percentage Improvement (Heuristic vs Eco-Routing)", fontsize=16, fontweight='bold')
ax.set_ylabel("Improvement (%)")
ax.set_xlabel("")
plt.xticks(rotation=45, ha='right')

# Add numeric labels on top of each bar.
for idx, row in rel_summary_plot_df.iterrows():
    val = row["Average Improvement (%)"]
    ax.text(idx, val + (1.5 if val >= 0 else -1.5),
            f"{val:.1f}%", ha='center',
            va='bottom' if val >= 0 else 'top',
            fontsize=11, fontweight='bold')
plt.tight_layout()
plt.show()
